### スクレイピング

In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2
from psycopg2 import sql

# スクレイピング関数
def scrape_transit_data(start_station, end_station):
    url = f"https://transit.yahoo.co.jp/search/result?from={start_station}&to={end_station}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    routes = []
    for route in soup.select('.routeDetail'):
        try:
            # 出発時刻と到着時刻
            times = route.select_one('.time').get_text().split('→')
            departure_time = times[0].strip()
            arrival_time = times[1].strip()
            
            # 乗り換え回数
            transfer_count = int(route.select_one('.transfer').get_text().replace('乗り換え', '').replace('回', ''))
            
            # 所要時間
            travel_time = route.select_one('.time').get_text().strip().split('(')[1].replace(')', '').replace('分', '')
            travel_time = int(travel_time)
            
            # 運賃
            fare = route.select_one('.fare').get_text().replace('¥', '').replace(',', '')
            fare = int(fare)
            
            # データをリストに追加
            routes.append({
                'departure_time': departure_time,
                'arrival_time': arrival_time,
                'transfer_count': transfer_count,
                'travel_time': travel_time,
                'fare': fare
            })
        except Exception as e:
            print(f"Error parsing route: {e}")
            continue

    return routes


### データベースに保存

In [ ]:
# データベース接続設定
conn = psycopg2.connect(
    dbname="your_database_name",
    user="your_username",
    password="your_password",
    host="your_host",
    port="your_port"
)
cursor = conn.cursor()

# テーブルの作成
def create_table():
    create_table_query = """
    CREATE TABLE IF NOT EXISTS routes_table (
        id SERIAL PRIMARY KEY,
        departure_time VARCHAR(10),
        arrival_time VARCHAR(10),
        transfer_count INTEGER,
        travel_time INTEGER,
        fare INTEGER
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

# データの挿入
def insert_data(routes):
    insert_query = """
    INSERT INTO routes_table (departure_time, arrival_time, transfer_count, travel_time, fare)
    VALUES (%s, %s, %s, %s, %s);
    """
    for route in routes:
        cursor.execute(insert_query, (
            route['departure_time'],
            route['arrival_time'],
            route['transfer_count'],
            route['travel_time'],
            route['fare']
        ))
    conn.commit()

# テーブル作成の実行
create_table()

# 川口駅から国際展示場駅までのデータ取得と保存
routes = scrape_transit_data("川口", "国際展示場")
insert_data(routes)

# データベース接続を閉じる
cursor.close()
conn.close()


### データの可視化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2

# データベース接続設定
conn = psycopg2.connect(
    dbname="your_database_name",
    user="your_username",
    password="your_password",
    host="your_host",
    port="your_port"
)

# データベースからデータを取得
query = """
SELECT departure_time, arrival_time, transfer_count, travel_time, fare
FROM routes_table
"""
data = pd.read_sql_query(query, conn)

# データの前処理
# 所要時間や運賃の型を確認・変換
data['travel_time'] = data['travel_time'].astype(int)
data['fare'] = data['fare'].astype(int)

# 出発時刻から時間帯を分類
def classify_time_period(departure_time):
    hour = int(departure_time.split(':')[0])
    if 6 <= hour < 12:
        return '朝'
    elif 12 <= hour < 18:
        return '昼'
    elif 18 <= hour < 24:
        return '夜'
    else:
        return '深夜'

data['time_period'] = data['departure_time'].apply(classify_time_period)

# **仮説1: 乗り換え回数と所要時間・運賃の関係**

# 乗り換え回数と所要時間の関係
plt.figure(figsize=(10, 6))
sns.boxplot(x='transfer_count', y='travel_time', data=data, palette='Set2')
plt.title('乗り換え回数と所要時間の関係', fontsize=16)
plt.xlabel('乗り換え回数', fontsize=14)
plt.ylabel('所要時間（分）', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 乗り換え回数と運賃の関係
plt.figure(figsize=(10, 6))
sns.boxplot(x='transfer_count', y='fare', data=data, palette='Set3')
plt.title('乗り換え回数と運賃の関係', fontsize=16)
plt.xlabel('乗り換え回数', fontsize=14)
plt.ylabel('運賃（円）', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# **仮説2: 時間帯（朝・昼・夜）と所要時間の関係**

# 時間帯と所要時間の関係
plt.figure(figsize=(10, 6))
sns.boxplot(x='time_period', y='travel_time', data=data, palette='Set1')
plt.title('時間帯と所要時間の関係', fontsize=16)
plt.xlabel('時間帯', fontsize=14)
plt.ylabel('所要時間（分）', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# 時間帯と運賃の関係
plt.figure(figsize=(10, 6))
sns.boxplot(x='time_period', y='fare', data=data, palette='coolwarm')
plt.title('時間帯と運賃の関係', fontsize=16)
plt.xlabel('時間帯', fontsize=14)
plt.ylabel('運賃（円）', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# データベース接続を閉じる
conn.close()
